In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[6],4
commitnumber,1013f91,a06b618
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[3]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312        1 -1.062290e-05                    1 -1.986769e-05
0.000750        2 -1.059140e-05                    2 -3.830843e-05
0.001052        3 -1.050866e-05                    3 -1.210745e-05
0.001476        4 -1.041045e-05                    4 -8.218825e-06
0.002070        5 -1.030669e-05                    5 -9.112775e-06
0.002904        6 -1.018848e-05                    6 -8.233626e-06
0.004074        7 -1.006336e-05                    7 -8.781505e-06
0.005714        8 -9.925573e-06                    8 -8.258278e-06
0.008015        9 -9.775611e-06                    9 -8.567241e-06
0.011243       10 -9.615750e-06                   10 -8.345542e-06
0.015771       11 -9.442848e-06                   11 -8.334586e-06
0.022122       12 -9.257462e-06                   12 -8.381514e-06
0.031031       13 -9.059223e-06                   13 -8.410998e-06
0.043528       14 -8.832380e-06                   14 -1.092587e-05
0.061057       15 -8.069688e-06                   15 -1.336694e-05
0.085645       16 -5.208693e-06                   16 -1.081103e-05
0.120136       17  1.535458e-06                   17 -6.514044e-06
0.168516       18  1.316966e-05                   18 -2.115342e-06
0.236378       19  3.193888e-05                   19  4.996025e-06
0.331549       20  6.115555e-05                   20  1.624757e-05
0.465100       21  1.054301e-04                   21  3.369214e-05
0.652400       22  1.706203e-04                   22  6.238974e-05
0.915100       23  2.419800e-04                   23  9.447179e-05
1.283650       24  2.483458e-04                   24  9.623544e-05
1.800600       25  1.899922e-04                   25  7.082261e-05
2.525700       26  1.311518e-04                   26  4.629964e-05
3.542800       27  8.592346e-05                   27  2.774766e-05
4.969550       28  5.357845e-05                   28  1.471693e-05
6.970850       29  3.166481e-05                   29  6.019741e-06
9.778100       30  1.684127e-05                   30  2.062853e-07
13.715850      31  6.401899e-06                   31 -3.865392e-06
19.239350      32 -8.125350e-07                   32 -6.643945e-06
26.987250      33 -5.081734e-06                   33 -8.310689e-06
37.855300      34 -7.531344e-06                   34 -9.362414e-06
53.100050      35 -9.347076e-06                   35 -1.026594e-05
73.887500      36 -1.072693e-05                   36 -1.111240e-05
97.662500      37 -1.150162e-05                   37 -1.176748e-05
121.437500     38 -1.191089e-05                   38 -1.219887e-05
145.212500     39 -1.228885e-05                   39 -1.242579e-05
168.987500     40 -1.297994e-05                   40 -1.323118e-05
192.762500     41 -1.891988e-05                   41 -2.014379e-05
216.537500     42 -4.201969e-05                   42 -5.172415e-05
240.312500     43 -6.550690e-05                   43 -1.045122e-04
264.087500     44 -5.472968e-05                   44 -1.671000e-04
287.862500     45  6.201903e-05                   45 -1.991180e-04
311.637500     46  3.413592e-04                   46 -1.003855e-04
335.412500     47  8.052359e-04                   47  1.490064e-04
359.187500     48  1.520792e-03                   48  5.941754e-04
382.962500     49  2.554054e-03                   49  1.313437e-03
406.737500     50  3.967896e-03                   50  2.381190e-03
430.512500     51  5.816758e-03                   51  3.857932e-03
454.287500     52  7.985431e-03                   52  5.722774e-03
478.062500     53  1.052540e-02                   53  8.040106e-03
501.837500     54  1.353775e-02                   54  1.090630e-02
525.612500     55  1.706594e-02                   55  1.424801e-02
549.387500     56  2.115224e-02                   56  1.821816e-02
573.162500     57  2.585836e-02                   57  2.28722

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -60.494101  0.000000e+00 -60.494101 -60.533504   
0.000624    2     -60.494101  6.370326e-10 -60.494101 -60.533504   
0.000876    3     -60.494101  8.972731e-10 -60.494101 -60.533504   
0.001229    4     -60.494101  1.273427e-09 -60.494101 -60.533504   
0.001723    5     -60.494101  1.818278e-09 -60.494101 -60.533504   
0.002417    6     -60.494101  2.607369e-09 -60.494101 -60.533504   
0.003391    7     -60.494101  3.750226e-09 -60.494101 -60.533504   
0.004757    8     -60.494101  5.405353e-09 -60.494101 -60.533504   
0.006672    9     -60.494101  7.802086e-09 -60.494101 -60.533504   
0.009359    10    -60.494101  1.127264e-08 -60.494101 -60.533504   
0.013128    11    -60.494101  1.629780e-08 -60.494101 -60.533504   
0.018415    12    -60.494101  2.357364e-08 -60.494101 -60.533504   
0.025830    13    -60.494101  3.410766e-08 -60.494101 -60.533504   
0.036232    14    -60.494101  4.935855e-08 -60.494101 -60.533504   
0.050823    15    -60.494102  7.148286e-08 -60.494101 -60.533504   
0.071291    16    -60.494102  1.057322e-07 -60.494101 -60.533504   
0.100000    17    -60.494102  1.681206e-07 -60.494101 -60.533504   
0.140271    18    -60.494102  2.941033e-07 -60.494101 -60.533504   
0.196760    19    -60.494102  5.492817e-07 -60.494101 -60.533505   
0.275997    20    -60.494102  1.063903e-06 -60.494101 -60.533505   
0.387100    21    -60.494102  2.096643e-06 -60.494100 -60.533505   
0.543100    22    -60.494103  4.161758e-06 -60.494098 -60.533506   
0.761700    23    -60.494102  8.253091e-06 -60.494094 -60.533506   
1.068500    24    -60.494101  1.575926e-05 -60.494085 -60.533506   
1.498800    25    -60.494099  2.650447e-05 -60.494073 -60.533506   
2.102400    26    -60.494097  3.875546e-05 -60.494059 -60.533506   
2.949000    27    -60.494098  5.182260e-05 -60.494046 -60.533508   
4.136600    28    -60.494100  6.552132e-05 -60.494034 -60.533510   
5.802500    29    -60.494103  7.984694e-05 -60.494023 -60.533514   
8.139200    30    -60.494110  9.501035e-05 -60.494015 -60.533519   
11.417000   31    -60.494119  1.112812e-04 -60.494008 -60.533527   
16.014700   32    -60.494134  1.287618e-04 -60.494004 -60.533538   
22.464000   33    -60.494153  1.475854e-04 -60.494006 -60.533553   
31.510500   34    -60.494180  1.687787e-04 -60.494011 -60.533573   
44.200100   35    -60.494216  1.940433e-04 -60.494022 -60.533601   
62.000000   36    -60.494266  2.246327e-04 -60.494042 -60.533639   
85.775000   37    -60.494333  2.604892e-04 -60.494072 -60.533692   
109.550000  38    -60.494399  2.942958e-04 -60.494104 -60.533746   
133.325000  39    -60.494466  3.280904e-04 -60.494138 -60.533802   
157.100000  40    -60.494535  3.620113e-04 -60.494173 -60.533858   
180.875000  41    -60.494608  3.987956e-04 -60.494209 -60.533919   
204.650000  42    -60.494727  4.650772e-04 -60.494262 -60.534019   
228.425000  43    -60.495072  6.915167e-04 -60.494380 -60.534314   
252.200000  44    -60.495910  1.344191e-03 -60.494565 -60.535050   
275.975000  45    -60.497675  2.955661e-03 -60.494719 -60.536629   
299.750000  46    -60.500998  6.453689e-03 -60.494545 -60.539643   
323.525000  47    -60.505838  1.225408e-02 -60.493583 -60.544087   
347.300000  48    -60.512261  2.094584e-02 -60.491315 -60.550003   
371.075000  49    -60.520563  3.353065e-02 -60.487032 -60.557697   
394.850000  50    -60.531044  5.120445e-02 -60.479839 -60.567507   
418.625000  51    -60.544000  7.533618e-02 -60.468664 -60.579710   
442.400000  52    -60.559727  1.074445e-01 -60.452282 -60.594535   
466.175000  53    -60.578122  1.483289e-01 -60.429793 -60.611948   
489.950000  54    -60.599205  1.990556e-01 -60.400150 -60.632028   
513.725000  55    -60.623149  2.611249e-01 -60.362024 -60.654994   
537.500000  56    -60.650098  3.361374e-01 -60.313960 -60.680805   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')